# Minimize beam size and measure emittance on DIAG0

In [ ]:
# set up env for running on SLAC production servers
import os
os.environ['OMP_NUM_THREADS']=str(6)

In [ ]:
run_dir = '/home/physics/ml_tuning/20241114_DIAG0'

# Read pv info from YAML files

In [ ]:
import sys
import yaml
# sys.path.append("../../")
sys.path.append("../")

from common import get_pv_objects, save_reference_point, set_magnet_strengths, \
    measure_pvs

In [ ]:
pv_bounds = yaml.safe_load(open("../../pv_bounds.yml"))
pv_objects = get_pv_objects("../../tracked_pvs.yml")

In [ ]:
pv_bounds

In [ ]:
reference = yaml.safe_load(open("reference.yml"))

def reset_pvs():
    set_magnet_strengths(reference, pv_objects, validate=False)

In [ ]:
reference

In [ ]:
reset_pvs()

In [1]:
from lcls_tools.common.devices.reader import create_screen, create_magnet
from lcls_tools.common.measurements.screen_profile import ScreenBeamProfileMeasurement

screen = create_screen(area='DIAG0', name='OTRDG02')
beamsize_measurement = ScreenBeamProfileMeasurement(device=screen)

ModuleNotFoundError: No module named 'h5py'

# Test screen measurement

In [ ]:
from matplotlib import pyplot as plt
results = beamsize_measurement.measure()
plt.imshow(results["raw_image"])
plt.imshow(results["processed_image"])

# Quad Scan Emittance

In [ ]:
magnet = create_magnet(area="DIAG0", name="QDG009")
magnet.metadata.length = 1.0 # placeholder value
energy = 3.0e9 # placeholder value
scan_values = [-6.0, -3.0, -0.0] # placeholder values

quad_scan = QuadScanEmittance(
    beamline = 'SC_DIAG0',
    energy = energy, 
    magnet = magnet,
    scan_values = scan_values, 
    beamsize_measurement = beamsize_measurement,
    design_twiss = ,
    rmat = ,

)

result_dict = quad_scan.measure()

In [ ]:
result